# Experiments Notebook

### This Jupyter Notebook contains all executed experiments and allows for reproduction of the results.

<span style="color:#FFC000">Note: Make sure you have completed setting up the development environment, as described in the *README.md*, as well as prepared the dataset by walking through the *setup.ipynb* notebook. Then make sure to select the local venv as kernel.</span>

---

### Path manipulation

To allow package imports, please run this cell below to add the <b>uav</b> package to path.

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

### Seed Generation

The script below will generate n seeds using secure random number generation and md5 hashing

In [ ]:
import secrets
import hashlib

n = 7

for i in range(n):
    randnum = secrets.randbelow(100_000_000_000)
    seed_hex = hashlib.md5(str(randnum).encode()).hexdigest()
    seed_int = int(seed_hex, 16) % 2**32 - 1
    print(seed_int)

4140620135
1132905197
3754523883
3492508408
3378715402
4213948446
1521997286
1377995568
1449566991


### Configuration
The following cells allows configuration of the experiments

In [2]:
SEEDS = [    
4140620135, # StratifiedShuffleSplit
1132905197, # RepeatedStratifiedKFold
3754523883, # |
3492508408, # |
3378715402, # Model Seeds
4213948446, # |
1521997286, # |
]

EXPERIMENT_NAMES = [
    "VISIBLE",
    "INFRARED",
    "VISIBLE_INFRARED"
]

# RSKF
N_SPLITS = 5
N_REPEATS = 5

FOLD_SIZE = 200

### Experiment Creation

The following cell will utilize the scikit-learn library to subsample the dataset (stratified) and then create the fold indices for data splitting in the experiments. 

They will be stored in <b>experiments/rskf_splits.npy</b>.

In [4]:
from uav.experiments.create import create_experiments

create_experiments(
    seeds=SEEDS[:2],
    source_dir="../datasets/anti-uav300",
    target_filepath = "../experiments/rskf_splits.npy",
    fold_size=FOLD_SIZE,
    n_splits=N_SPLITS, 
    n_repeats=N_REPEATS,
    )

Processing Sequences: 100%|██████████| 318/318 [00:10<00:00, 31.25it/s]


Original X shape: (296901,)
Original stratification labels: (array([0, 1]), array([ 16683, 280218]))
Sampled X shape: (1000,)
Sampled stratification labels: (array([0, 1]), array([ 56, 944]))
Saved RSKF config to ../experiments/rskf_splits.npy.


### Experiment Execution

The following cell will run all training experiments one by one and store the metric results of each single training as a row in <b>metrics.txt</b>. 

The script automatically resumes training in case of interruptions and cleans partially completed sessions.

In [ ]:
from uav.experiments.run import run_experiments

run_experiments(
    experiment_rskf_file_npy="../experiments/rskf_splits.npy",
    metrics_file_txt="../results/metrics.csv",
    model_seeds=SEEDS[2:7],
    epochs=20,
    model_weight_path="../models/weights/yolo12n.pt",
    run_dir="../results/exp_train_runs"
    )


Processing fold 0
Running vz-3754523883 ...
New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.219 🚀 Python-3.13.5 torch-2.9.0 MPS (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/var/folders/xw/45g67_dd7glf1l755cb4gtgc0000gn/T/tmpdnie_hyd/cfg.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../models/weights/yolo12n.pt, momentu